In [7]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.nn import LayerNorm
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.nn import TransformerEncoder, TransformerDecoder, TransformerEncoderLayer, TransformerDecoderLayer
import torch.optim as optim

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mediapipe as mp
import random
import time
import cv2

# データの読み込みと前処理
def preprocess_data(csv_path):
    df = pd.read_csv(csv_path)
    x_data = []
    y_data = []

    for i in range(len(df) - n_seq):
        x_sequence = df.iloc[i:i+n_seq][[f'{j}_{c}' for j in range(num_joints) for c in ['x', 'y', 'z']]].values
        x_data.append(x_sequence)

        y_sequence = df.iloc[i+n_seq][[f'{j}_{c}' for j in range(num_joints) for c in ['x', 'y', 'z']]].values
        y_data.append(y_sequence)

    x_data = np.array(x_data, dtype=np.float32)
    y_data = np.array(y_data, dtype=np.float32)

    return x_data, y_data

# じゃんけんの手のラベル
janken_labels = {0: 'チョキ', 1: 'グー', 2: 'パー'}

# DataLoaderの使用
def create_dataloader(x_train, y_train, batch_size):
    train_dataset = TensorDataset(torch.tensor(x_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    return train_loader


# PositionalEncodingクラスを定義
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=0.1)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

# データが numpy.ndarray または torch.Tensor の場合に対応する関数
def squeeze_dims_if_needed(data):
    if isinstance(data, np.ndarray):
        if data.ndim == 3 and data.shape[1] == 1:
            data = np.squeeze(data, axis=1)
    elif isinstance(data, torch.Tensor):
        if data.dim() == 3 and data.size(1) == 1:
            data = data.squeeze(1)
    return data

# numpy.ndarray を torch.Tensor に変換する関数
def to_tensor(data):
    if isinstance(data, np.ndarray):
        data = torch.tensor(data)
    return data

# データの読み込みと前処理
#train_csv_path = 'test_10/choki_test_10/choki_test.csv'
train_csv_path = '2_19_hand.csv'
test_csv_path = 'test_10/choki_test_10/choki_test.csv'

n_seq = 30  # 例として30を指定
num_joints = 17  # 例として17を指定

# x_train, y_train が numpy.ndarray または torch.Tensor であることを想定
x_train, y_train = preprocess_data(train_csv_path)
x_test, y_test = preprocess_data(test_csv_path)

x_train = squeeze_dims_if_needed(x_train)
y_train = squeeze_dims_if_needed(y_train)

# numpy.ndarray を torch.Tensor に変換
x_train = to_tensor(x_train)
y_train = to_tensor(y_train)

# x_train と y_train が 2 次元であることを確認
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")

# データの連結
try:
    input_y_train = torch.cat((x_train[:, -1:], y_train[:, :-1]), dim=1)
    print(f"input_y_train shape: {input_y_train.shape}")
except IndexError as e:
    print(f"Error during concatenation: {e}")

# 以下のコードは、マスクの作成やモデルの出力を計算する部分
# 例としてダミーの create_mask 関数と model 関数を定義
def create_mask(x, y):
    return torch.ones_like(x), torch.ones_like(y)  # ダミーのマスク生成

# ダミーのモデルクラス
class DummyModel:
    def __call__(self, x_train, y_train, mask_x_train, mask_y_train):
        return x_train + y_train  # ダミーの出力計算

model = DummyModel()

# マスクの作成
mask_x_train, mask_y_train = create_mask(x_train, input_y_train)

# モデルの出力を計算
output = model(x_train=x_train, y_train=input_y_train, mask_x_train=mask_x_train, mask_y_train=mask_y_train)
print(f"Output shape: {output.shape}")


x_train shape: torch.Size([2376, 30, 51])
y_train shape: torch.Size([2376, 51])


RuntimeError: Tensors must have same number of dimensions: got 3 and 2